# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.


In [135]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:


In [136]:
from secrets import IEX_CLOUD_API_TOKEN
stocks = pd.read_csv('sp_500_stocks.csv')
stocks


,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
496,YUM
497,ZBH
498,ZBRA
499,ZION


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.


In [137]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2442427795111,
 'week52high': 187.53,
 'week52low': 129.02,
 'week52highSplitAdjustOnly': 189.66,
 'week52lowSplitAdjustOnly': 132.91,
 'week52change': 0.05748992227298021,
 'sharesOutstanding': 16148479185,
 'float': 0,
 'avg10Volume': 105481193,
 'avg30Volume': 98860173,
 'day200MovingAvg': 156.1,
 'day50MovingAvg': 167.95,
 'employees': 153622,
 'ttmEPS': 6.19,
 'ttmDividendRate': 0.9130405623897275,
 'dividendYield': 0.006151485070695266,
 'nextDividendDate': '',
 'exDividendDate': '2022-07-23',
 'nextEarningsDate': '2022-10-18',
 'peRatio': 24.691152989874972,
 'beta': 1.2746148296691535,
 'maxChangePercent': 57.23763632885126,
 'year5ChangePercent': 3.22448852891551,
 'year2ChangePercent': 0.3338600220000331,
 'year1ChangePercent': 0.06331565167264376,
 'ytdChangePercent': -0.15792541098633367,
 'month6ChangePercent': -0.1469666996333157,
 'month3ChangePercent': 0.09408915689889971,
 'month1ChangePercent': -0.08424109953546224,
 'day30Ch

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.


In [138]:
data['year1ChangePercent']


0.06331565167264376

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.


In [139]:
# Function sourced from
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Ticker', 'Price',
              'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.


In [140]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=price,stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ],
                index=my_columns),
            ignore_index=True)

final_dataframe


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,127.200,-0.222062,N/A
1,AAL,12.450,-0.40626,N/A
2,AAP,167.502,-0.221155,N/A
3,AAPL,146.334,0.064186,N/A
4,ABBV,148.400,0.413808,N/A
...,...,...,...,...
496,YUM,108.860,-0.104887,N/A
497,ZBH,111.110,-0.29199,N/A
498,ZBRA,277.186,-0.501143,N/A
499,ZION,53.100,-0.13214,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [141]:
final_dataframe.sort_values('One-Year Price Return',
                            ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop=True, inplace=True)
final_dataframe


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,81.570,2.356446,N/A
1,OXY,64.480,1.079799,N/A
2,HRB,44.657,0.850615,N/A
3,CF,98.510,0.831174,N/A
4,DVN,63.440,0.813744,N/A
5,MCK,353.520,0.760814,N/A
6,MRO,23.140,0.67985,N/A
7,UNM,39.420,0.671971,N/A
8,MPC,98.850,0.65648,N/A
9,APA,35.170,0.644911,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!


In [142]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')

    try:
        float(portfolio_size)
    except:
        print('That is not a number! \n Please try again:')
        portfolio_size = input("Enter the value of your portfolio:")


portfolio_input()
print(portfolio_size)


1000000


In [143]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(
        position_size / final_dataframe.loc[i, 'Price']
    )
final_dataframe


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,81.570,2.356446,245
1,OXY,64.480,1.079799,310
2,HRB,44.657,0.850615,447
3,CF,98.510,0.831174,203
4,DVN,63.440,0.813744,315
5,MCK,353.520,0.760814,56
6,MRO,23.140,0.67985,864
7,UNM,39.420,0.671971,507
8,MPC,98.850,0.65648,202
9,APA,35.170,0.644911,568


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

- High-quality momentum stocks show "slow and steady" outperformance over long periods of time
- Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of:

- 1-month price returns
- 3-month price returns
- 6-month price returns
- 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.


In [144]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=price,stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series([symbol,
                       data[symbol]['price'],
                       'N/A',
                       data[symbol]['stats']['year1ChangePercent'],
                       'N/A',
                       data[symbol]['stats']['month6ChangePercent'],
                       'N/A',
                       data[symbol]['stats']['month3ChangePercent'],
                       'N/A',
                       data[symbol]['stats']['month1ChangePercent'],
                       'N/A',
                       'N/A'
                       ],
                      index=hqm_columns),
            ignore_index=True)

hqm_dataframe.columns
hqm_dataframe


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,129.230,N/A,-0.227525,N/A,-0.086674,N/A,0.059957,N/A,-0.047373,N/A,N/A
1,AAL,12.860,N/A,-0.420198,N/A,-0.268822,N/A,-0.056855,N/A,-0.073762,N/A,N/A
2,AAP,167.213,N/A,-0.223035,N/A,-0.21266,N/A,-0.081305,N/A,-0.051208,N/A,N/A
3,AAPL,147.021,N/A,0.062833,N/A,-0.146148,N/A,0.096196,N/A,-0.08815,N/A,N/A
4,ABBV,146.785,N/A,0.414006,N/A,-0.092247,N/A,-0.043372,N/A,0.062908,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,109.490,N/A,-0.10608,N/A,-0.100204,N/A,-0.042597,N/A,-0.036336,N/A,N/A
497,ZBH,108.230,N/A,-0.296219,N/A,-0.142516,N/A,0.010153,N/A,0.000195,N/A,N/A
498,ZBRA,266.262,N/A,-0.49562,N/A,-0.361385,N/A,-0.104969,N/A,-0.122662,N/A,N/A
499,ZION,54.530,N/A,-0.133289,N/A,-0.230819,N/A,0.018697,N/A,-0.057176,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

- `One-Year Price Return`
- `Six-Month Price Return`
- `Three-Month Price Return`
- `One-Month Price Return`

Here's how we'll do this:


In [145]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        if hqm_dataframe.loc[row, change_col] == None:
            hqm_dataframe.loc[row, change_col] = 0.0

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(
            hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])
hqm_dataframe


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,129.230,N/A,-0.227525,34.331337,-0.086674,68.063872,0.059957,81.437126,-0.047373,75.449102,N/A
1,AAL,12.860,N/A,-0.420198,8.982036,-0.268822,24.550898,-0.056855,40.319361,-0.073762,60.878244,N/A
2,AAP,167.213,N/A,-0.223035,35.129741,-0.21266,39.92016,-0.081305,28.542914,-0.051208,73.652695,N/A
3,AAPL,147.021,N/A,0.062833,77.644711,-0.146148,54.091816,0.096196,88.423154,-0.08815,49.301397,N/A
4,ABBV,146.785,N/A,0.414006,95.209581,-0.092247,66.666667,-0.043372,45.908184,0.062908,98.003992,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,109.490,N/A,-0.10608,53.692615,-0.100204,64.271457,-0.042597,46.307385,-0.036336,80.439122,N/A
497,ZBH,108.230,N/A,-0.296219,22.355289,-0.142516,54.89022,0.010153,67.664671,0.000195,92.215569,N/A
498,ZBRA,266.262,N/A,-0.49562,3.992016,-0.361385,7.58483,-0.104969,22.55489,-0.122662,24.351297,N/A
499,ZION,54.530,N/A,-0.133289,50.0998,-0.230819,35.728543,0.018697,70.05988,-0.057176,70.05988,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.


In [146]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(
            hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.


In [147]:
hqm_dataframe.sort_values(by='HQM Score', ascending=False)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,129.230,N/A,-0.227525,34.331337,-0.086674,68.063872,0.059957,81.437126,-0.047373,75.449102,64.820359
1,AAL,12.860,N/A,-0.420198,8.982036,-0.268822,24.550898,-0.056855,40.319361,-0.073762,60.878244,33.682635
2,AAP,167.213,N/A,-0.223035,35.129741,-0.21266,39.92016,-0.081305,28.542914,-0.051208,73.652695,44.311377
3,AAPL,147.021,N/A,0.062833,77.644711,-0.146148,54.091816,0.096196,88.423154,-0.08815,49.301397,67.365269
4,ABBV,146.785,N/A,0.414006,95.209581,-0.092247,66.666667,-0.043372,45.908184,0.062908,98.003992,76.447106
5,ABC,139.892,N/A,0.161479,85.429142,-0.100114,64.471058,-0.030645,51.696607,-0.053916,71.656687,68.313373
6,ABMD,259.660,N/A,-0.256586,28.542914,-0.224477,37.52495,0.003963,65.668663,-0.032022,82.235529,53.493014
7,ABT,101.550,N/A,-0.16172,44.710579,-0.17617,47.704591,-0.074286,32.135729,-0.032517,82.035928,51.646707
8,ACN,260.820,N/A,-0.190784,39.720559,-0.203226,42.914172,-0.074127,32.335329,-0.125897,23.153693,34.530938
9,ADBE,281.500,N/A,-0.526368,2.794411,-0.38304,6.586826,-0.240623,3.592814,-0.272546,0.598802,3.393214


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.


In [148]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")


portfolio_input()
print(portfolio_size)


1000000


In [149]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(
        position_size / final_dataframe['Price'][i])
final_dataframe

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,81.570,2.356446,245
1,OXY,64.480,1.079799,310
2,HRB,44.657,0.850615,447
3,CF,98.510,0.831174,203
4,DVN,63.440,0.813744,315
5,MCK,353.520,0.760814,56
6,MRO,23.140,0.67985,864
7,UNM,39.420,0.671971,507
8,MPC,98.850,0.65648,202
9,APA,35.170,0.644911,568


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.


In [150]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index=False)


## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

- String format for tickers
- \$XX.XX format for stock prices
- \$XX,XXX format for market capitalization
- Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.


In [151]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_template = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_template = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

percent_template = writer.book.add_format(
    {
        'num_format': '0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)


In [152]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', integer_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(
        f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(
        f'{column}1', column_formats[column][0], string_template)


## Saving Our Excel Output

As before, saving our Excel output is very easy:


In [153]:
writer.save()
